# Exercício 9_10: Agente ReAct

**Nome:** Caio Petrucci dos Santos Rosa

**RA:** 248245

## Enunciado

- Implementar a ReAct  com LLaMa 3 70B (groq)

- Testar no dataset do IIRC - 50 primeiras perguntas com resposta (test_questions.json em anexo)

- Usar o prompt do LLaMAIndex: https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py

- Salvar as respostas finais das 50 perguntas no JSON para exercício futuro de avaliação

- Instruir o modelo a seguir a sequência Thougth, Action, Input, Observation (a observação não é do próprio modelo, mas resultado da busca)

- É necessário usar o parâmetro stop_sequence="Observation:", para o modelo parar de gerar texto e esperar o retorno da busca. Implementem o código da busca e retornem os top-k documentos pro modelo (sugestão: k=5).

- Instruir o modelo agir passo-a-passo (decomposição da pergunta).

- Podem usar o LangChain, LLaMAindex ou outro framework. Ou implementar na mão.

- Usar a busca como ferramenta

- Usar o BM25 como buscador (repetir indexação do exercício passado)

- Usar a indexação do Visconde: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb

# Bibliotecas e pacotes

In [1]:
!apt install openjdk-21-jdk-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-21-jdk-headless is already the newest version (21.0.2+13-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-21-openjdk-amd64'

In [3]:
!pip install pyserini faiss-cpu
!pip install -q groq
!pip install -q beautifulsoup4
!pip install langchain
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompat

In [4]:
from google.colab import userdata, drive
from groq import Groq, RateLimitError
from tqdm import tqdm
from bs4 import BeautifulSoup
from pyserini.search.lucene import LuceneSearcher
from collections import Counter
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

import json
import threading
import time
import json
import spacy
import argparse
import collections
import numpy as np
import re
import string
import sys
import unicodedata
import pandas as pd

import warnings
warnings.simplefilter('ignore')

# Atributos e hiper-parâmetros

In [5]:
LLM_MODEL_NAME = "llama3-70b-8192"
LLM_TEMPERATURE = 0

DOCUMENT_WINDOW_STRIDE = 3
DOCUMENT_WINDOW_SIZE = 2

RETRIEVER_TOP_K = 5

N_QUESTIONS = 50

In [6]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [40]:
# Prompt adaptado a partir da combinação dos seguintes prompts:
#   - https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py
#   - https://smith.langchain.com/hub/hwchase17/react-chat (entrei em contato através do código do Fábio Grassiotto)

REACT_CHAT_SYSTEM_HEADER = """\

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.
You are a Large Language Model trained by Meta AI.
As a language model, you are able to generate human-like text based on the input you receive, allowing yourself to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions.
Additionally, you are able to generate your own text based on the input it receives, allowing yourself to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, you are a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the user needs help with a specific question or just want to have a conversation about a particular topic, you are here to assist.

TOOLS:
------
You have access to a wide variety of tools.
You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.
You have access to the following tools:
{tools}

To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Your final answer must be short, no more than 10 words, and use numerals instead of words for numbers.
If you don't know any plausible answer, answer "Not enough information provided in the documents."

Begin!
New input: {input}
{agent_scratchpad}
"""

# Dataset IIRC

In [41]:
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
!tar -xzf context_articles.tar.gz

--2024-05-16 02:12:15--  https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.92.224.34, 52.92.129.250, 52.92.164.210, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|52.92.224.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 385263479 (367M) [application/x-gzip]
Saving to: ‘context_articles.tar.gz.1’

context_articles.ta 100%[===================>] 367.42M  31.3MB/s    in 9.5s    

2024-05-16 02:12:25 (38.5 MB/s) - ‘context_articles.tar.gz.1’ saved [385263479/385263479]



In [42]:
test_questions = json.load(open('test_questions.json', 'r'))
context_articles = json.load(open('context_articles.json', 'r'))

# Indexação e pré-processamento

In [43]:
# Código adaptado do código do Fábio Grassiotto
# Também foi inspirado no código do Visconde (https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb)

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def extract_data(num_questions, dataset, context_articles):
    questions_found = []
    num_questions_found = 0
    documents = []
    all_titles = []

    for item in tqdm(dataset):
        question = item['question']
        answer = item['answer']
        answer_type = answer['type']

        if answer_type == 'binary' or answer_type == 'value':
            final_answer = answer['answer_value']
        elif answer_type == 'span':
            final_answer = answer['answer_spans'][0]['text']
        elif answer_type == 'none':
            final_answer = 'none'
        else:
            final_answer = 'An error perhaps, bad type'

        if final_answer == 'none':
            # Skip this one.
            continue
        else:
            # Thats a good question.

            # First add some extra info in the context-part
            for context in item['context']:
                if context['passage'] == "main":
                    # Cleanup html tags
                    clean_text = remove_html_tags(context['text'])
                    documents.append({
                        "title": item['title'].lower(),
                        "content": clean_text
                    })

            all_titles.append(item['title'].lower())

            # And then grab the text from the articles json
            for link in item["question_links"]:
                if link.lower() in context_articles and link.lower() not in all_titles:
                  # Cleanup html tags
                  soup = BeautifulSoup(context_articles[link.lower()], 'html.parser')
                  clean_text = soup.get_text()

                  documents.append({
                    "title": link,
                    "content": clean_text
                  })
                all_titles.append(link.lower())

            questions_found.append({"Question": question, "Answer": final_answer})
            num_questions_found += 1

            if num_questions_found == num_questions:
                # found our questions
                break

    return questions_found, documents, all_titles

In [44]:
questions_to_ask, documents, all_titles = extract_data(N_QUESTIONS, test_questions, context_articles)

 98%|█████████▊| 49/50 [00:02<00:00, 16.82it/s]


In [45]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

In [46]:
# Código baseado no código do Visconde (https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb)
# Pequena adaptação para o spaCy v3

def sliding_window_split(documents, stride, window_size):
    treated_documents = []

    for j, document in enumerate(tqdm(documents)):
        doc_text = document['content']
        doc = nlp(doc_text[:10000])
        sentences = [sent.text.strip() for sent in doc.sents]
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i+window_size]).strip()
            treated_documents.append({
                "title": document['title'],
                "contents": document['title']+ ". " + segment,
                "segment": segment
            })
            if i+window_size >= len(sentences):
                break

    return treated_documents

In [47]:
def add_id_and_filter_empty(documents):
    filtered_documents = []
    for i, doc in enumerate(documents):
        if doc['segment'] != "":
            filtered_doc = { **doc }
            filtered_doc['id'] = i
            filtered_documents.append(filtered_doc)
    return filtered_documents

In [48]:
treated_documents = add_id_and_filter_empty(sliding_window_split(documents, stride=DOCUMENT_WINDOW_STRIDE, window_size=DOCUMENT_WINDOW_SIZE))

100%|██████████| 124/124 [00:02<00:00, 45.28it/s]


In [49]:
!mkdir iirc_index_content

mkdir: cannot create directory ‘iirc_index_content’: File exists


In [50]:
with open("iirc_index_content/contents.jsonl",'w') as file:
    for doc in treated_documents:
        file.write(json.dumps(doc)+"\n")

In [51]:
!python3 -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 1 -input iirc_index_content -index iirc_index -storeRaw

2024-05-16 02:13:08,841 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-05-16 02:13:08,844 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-05-16 02:13:08,845 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-05-16 02:13:08,845 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: iirc_index_content
2024-05-16 02:13:08,846 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-05-16 02:13:08,846 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: iirc_index
2024-05-16 02:13:08,847 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 1
2024-05-16 02:13:08,847 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false
May 16, 2024 2:13:08 AM org.apac

# BM25 com PyseriniRetriever

In [52]:
class PyseriniRetriever:
    def __init__(self, top_k):
        self._searcher = LuceneSearcher('./iirc_index')
        self._top_k = top_k

    def __call__(self, query):
        hits = self._searcher.search(query, k=self._top_k)
        return [ json.loads(hit.lucene_document.get('raw')) for hit in hits ]

In [53]:
retriever = PyseriniRetriever(RETRIEVER_TOP_K)

In [54]:
@tool
def get_topk_with_bm25(question: str) -> list[str]:
    """Returns a sequence of five document passages with texts to help solve a question."""
    searched_documents = retriever(question)
    results = [ f"Document passage {i}: {doc['contents']}" for i, doc in enumerate(searched_documents, 1) ]
    return results

# Agente ReAct com LangChain

In [55]:
# Código adaptado do código do Fábio Grassiotto

def get_agent_executor(verbose=False):
    prompt = PromptTemplate.from_template(REACT_CHAT_SYSTEM_HEADER)
    llm = ChatGroq(
        temperature=LLM_TEMPERATURE,
        model_name=LLM_MODEL_NAME,
        api_key=GROQ_API_KEY
    )

    tools = [get_topk_with_bm25]
    agent = create_react_agent(llm, [get_topk_with_bm25], prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose, handle_parsing_errors=True)

    return agent_executor

In [56]:
agent_executor = get_agent_executor(verbose=True)

# Avaliação

In [57]:
# Código adaptado do código do Visconde (https://github.com/neuralmind-ai/visconde/blob/main/iirc_generate_and_evaluate.ipynb)

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    def remove_accents(input_str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        only_ascii = nfkd_form.encode('ASCII', 'ignore')
        return only_ascii.decode("utf-8")

    return white_space_fix(remove_articles(remove_punc(lower(remove_accents(s)))))


def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [58]:
# Código adaptado do código do Fábio Grassiotto

def evaluate_agent_executor(executor):
    df = pd.DataFrame(columns=['question', 'answer', 'LLM answer', 'F1', 'Exact Match'])

    for item in tqdm(questions_to_ask):
        question = item.get('Question')
        answer = item.get('Answer')

        agent_answer = executor.invoke({"input": "Question: " + question})
        llm_answer = agent_answer['output']
        f1_score = compute_f1(llm_answer, answer)
        e_match_score = compute_exact(llm_answer, answer)

        row = pd.Series(
            [question, answer, llm_answer, f1_score, e_match_score],
            index=df.columns
        )
        df = pd.concat(
            [df, pd.DataFrame([row])],
            ignore_index=True
        )

    return df

In [59]:
df = evaluate_agent_executor(agent_executor)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_topk_with_bm25
Action Input: What is Zeus know for in Greek mythology?['Document passage 1: Zeus. In the section of the Iliad known to scholars as the Deception of Zeus, the two of them are described as having begun their sexual relationship without their parents knowing about it. The conquests of Zeus among nymphs and the mythic mortal progenitors of Hellenic dynasties are famous.', "Document passage 2: Greek mythology. Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore. These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual practices.", "Document passage 3: Greek mythology. Greek mythology has changed over time to accommodate the evolution of their culture, of which mythology, both

  2%|▏         | 1/50 [00:01<01:22,  1.68s/it]

Thought: Do I need to use a tool? No
Final Answer: King of gods and god of sky and thunder.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_topk_with_bm25
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['Document passage 1: giovanni messe. he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'Document passage 2: Havana. Such attacks convinced the Spanish Crown to fund the construction of the first fortresses in the main cities – not only to counteract the pirates and corsairs, but also to exert more control over commerce with the West Indies, and to limit the extensive contrabando (black market) that had arisen due to the trade restrictions imposed by the Casa de Contratación of Seville (the crown-controlled trading house that held a monopoly on New World trade). Ships from all over the New World carried products first to Havana, in order to be t

  4%|▍         | 2/50 [00:03<01:30,  1.88s/it]

...

Thought: Do I need to use a tool? No
Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_topk_with_bm25
Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?['Document passage 1: william tecumseh sherman. Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', 'Document passage 2: Battle of Fort Henry. The Battle of Fort Henry was fought on February 6, 1862, in Donelson Middle Tennessee, during the American Civil War. It was the first important victory for the Union and Brig.', 'Document passage 3: Battle of Hanau. The French Grande Armée had suffered horrendous casualties at the battle of Leipzig, which left the French Corps at a fraction of its prior strength. Emperor Napoleon I was in personal command of the French forces in the battle.', 'Document passage 4: Arizona.

 98%|█████████▊| 49/50 [13:10<00:25, 25.93s/it]

Thought: Do I need to use a tool? No
Final Answer: Fort Henry had higher Union casualty rate.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_topk_with_bm25
Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?['Document passage 1: battle of mazocoba. Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', 'Document passage 2: Costa Rica. The impact of indigenous peoples on modern Costa Rican culture has been relatively small compared to other nations, since the country lacked a strong native civilization to begin with. Most of the native population was absorbed into the Spanish-speaking colonial society through inter-marriage, except for some small remnants, the most significant of which are the Bribri and Boruca tribes who still inhabit the mountains of the Cordillera de Talamanca, in

100%|██████████| 50/50 [13:29<00:00, 16.18s/it]

Thought: Do I need to use a tool? No
Final Answer: USA

> Finished chain.


In [65]:
df.head()

,question,answer,LLM answer,F1,Exact Match
0,What is Zeus know for in Greek mythology?,sky and thunder god,king of gods and god of sky and thunder,0.615385,0
1,How long had the First World War been over whe...,5,5,1.000000,1
2,How old was Messe when the First World War sta...,30,not enough information provided in documents,0.000000,0
3,How long had Angela Scoular been acting profes...,2,2 years,0.666667,0
4,What is the capacity of the stadium where Brun...,26688,26272,0.000000,0


In [64]:
print("      Metrics     ")
print("------------------")
print(f"F1 score:\n\tAvg: {df['F1'].mean():.2f}.\n\tStd: {df['F1'].std():.2f}.")
print(f"Exact Match score:\n\tAvg: {df['Exact Match'].mean():.2f}.\n\tStd: {df['Exact Match'].std():.2f}.")

      Metrics     
------------------
F1 score:
	Avg: 0.39.
	Std: 0.44.
Exact Match score:
	Avg: 0.28.
	Std: 0.45.
